In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import copy
import time

import sys
sys.path.append('../../')

from data.processing import get_data

import models
import random

from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new, forecast_top_k_trials
from main.seir.sensitivity import calculate_sensitivity_and_plot
import main.seir.uncertainty as uncertainty_module
from utils.generic.create_report import save_dict_and_create_report_simple
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles

import yaml

In [ ]:
predictions_dict = {}

In [ ]:
config_filename = 'experiments/undetected.yaml'
config = read_config(config_filename)

In [ ]:
output_folder = '../../misc/reports/{}'.format(datetime.datetime.now().strftime("%Y_%m%d_%H%M%S"))

## Perform M1 and M2 fits

### undetected testing model fitting

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(**copy.deepcopy(config['fitting'])) 

# m2_params = copy.deepcopy(config['fitting'])
# m2_params['split']['val_period'] = 0
# predictions_dict['m2'] = single_fitting_cycle(**m2_params)

predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")

In [ ]:
predictions_dict['m1']['best_params']

## Loss Dataframes

### M1 Loss DataFrame

In [ ]:
predictions_dict['m1']['df_loss']

In [ ]:
sum([0.25*i for i in predictions_dict['m1']['df_loss']['train'].to_list()])

## Plot Forecasts

In [ ]:
predictions_dict['m1']['forecasts'] = {}
predictions_dict['m1']['forecasts']['best'] = get_forecast(predictions_dict, train_fit='m1', 
                                                           model=config['fitting']['model'], 
                                                           days=config['forecast']['forecast_days'], tests_done=100000)

predictions_dict['m1']['plots']['forecast_best'] = plot_forecast(predictions_dict,
                                                                 (config['fitting']['data']['dataloading_params']['state'], 
                                                                  config['fitting']['data']['dataloading_params']['district']), 
                                                                 error_bars=True, train_fit='m1')

# predictions_dict['m1']['trials_processed'] = forecast_all_trials(predictions_dict, train_fit='m1', 
#                                                                  model=config['fitting']['model'], 
#                                                                  forecast_days=config['forecast']['forecast_days'], tests_done=10000)

# kforecasts = plot_top_k_trials(predictions_dict, train_fit='m1',
#                                k=config['forecast']['num_trials_to_plot'],
#                                which_compartments=config['forecast']['plot_topk_trials_for_columns'])
                               
# predictions_dict['m1']['plots']['forecasts_topk'] = {}
# for column in config['forecast']['plot_topk_trials_for_columns']:
#     predictions_dict['m1']['plots']['forecasts_topk'][column.name] = kforecasts[column]

## Create Report

In [ ]:
save_dict_and_create_report_simple(predictions_dict, config, ROOT_DIR=output_folder, config_filename=config_filename)

## Create Output

In [ ]:
df_output = create_decile_csv_new(predictions_dict)
df_output.to_csv(f'{output_folder}/deciles.csv')

In [ ]:
log_wandb(predictions_dict)

## Create All Trials Output

In [ ]:
df_all = create_all_trials_csv(predictions_dict)
df_all.to_csv(f'{output_folder}/all_trials.csv')